# RVX Data Catalog & Exploration

Comprehensive exploration of all datasets in the RVX folders:
- `traveling_survey/` - National travel survey data
- `zonal_register_data/` - Zonal statistical data (SDAT files)

In [1]:
import os
from pathlib import Path
import pandas as pd
import json
from datetime import datetime
from collections import defaultdict

# Setup
from synlab.utils import get_project_root

project_root = get_project_root()
rvx_path = project_root / 'data' / 'raw' / 'population' / 'rvx'

print(f"Project root: {project_root}")
print(f"RVX path: {rvx_path}")
print(f"Path exists: {rvx_path.exists()}")

Project root: /Users/anderskielland/Documents/Synthetic data/code/synthetic-lab
RVX path: /Users/anderskielland/Documents/Synthetic data/code/synthetic-lab/data/raw/population/rvx
Path exists: True


## 1. Folder Overview

List all files and their sizes in both folders.

In [15]:
def get_folder_structure(folder_path):
    """
    Walk through folder and collect all files with metadata.
    Returns list of dicts with file info.
    """
    files_info = []
    
    for root, dirs, files in os.walk(folder_path):
        # Skip .DS_Store and other system files
        files = [f for f in files if not f.startswith('.')]
        
        for file in files:
            file_path = Path(root) / file
            size_bytes = os.path.getsize(file_path)
            size_mb = size_bytes / (1024 * 1024)
            
            rel_path = file_path.relative_to(folder_path)
            
            files_info.append({
                'filename': file,
                'relative_path': str(rel_path),
                'full_path': str(file_path),
                'size_bytes': size_bytes,
                'size_mb': round(size_mb, 2),
                'extension': Path(file).suffix
            })
    
    return sorted(files_info, key=lambda x: x['filename'])

# Explore both folders
folders = ['traveling_survey', 'zonal_register_data']
all_files = {}

for folder_name in folders:
    folder_path = rvx_path / folder_name
    print(f"\n{'='*80}")
    print(f"📁 {folder_name.upper()}")
    print(f"{'='*80}")
    
    files = get_folder_structure(folder_path)
    all_files[folder_name] = files
    
    print(f"\nTotal files: {len(files)}")
    print(f"\nFile listing:")
    print("-" * 100)
    
    for f in files:
        print(f"{f['filename']:<60} {f['size_mb']:>10.2f} MB  {f['extension']}")
    
    # Summary by extension
    by_ext = defaultdict(int)
    for f in files:
        by_ext[f['extension']] += 1
    
    print(f"\nBy extension:")
    for ext, count in sorted(by_ext.items()):
        print(f"  {ext if ext else '[no ext]':<15} {count:>3} files")


📁 TRAVELING_SURVEY

Total files: 8

File listing:
----------------------------------------------------------------------------------------------------
Filemail.com - Nasjonal RVU akkumulert data.zip                  228.44 MB  .zip
Filemail.com - RVU 2025.zip                                      128.01 MB  .zip
Nasjonal_RVU_PERSON_Nov26_0901.sav                                63.65 MB  .sav
Nasjonal_RVU_REISER_Nov26_0901.sav                                64.36 MB  .sav
Oppdatert skjema RVU_2025.docx                                     0.37 MB  .docx
RVU 2019-2024 Personfil Vektet 251125.sav                         93.56 MB  .sav
RVU 2019_2024 Reisefil 251107.sav                                134.88 MB  .sav
Spørreskjema_RVU_2021_2024.docx                                    0.20 MB  .docx

By extension:
  .docx             2 files
  .sav              4 files
  .zip              2 files

📁 ZONAL_REGISTER_DATA

Total files: 28

File listing:
--------------------------------------------

## 2. Explore Data File Types

Understand the structure of different data formats (.dbf, .sav, .xlsx)

In [16]:
# Install required libraries for reading different formats
import subprocess
import sys

packages = ['pyreadstat', 'openpyxl', 'dbfread']

for package in packages:
    try:
        __import__(package)
        print(f"✓ {package} already installed")
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])
        print(f"✓ {package} installed")

✓ pyreadstat already installed
✓ openpyxl already installed
✓ dbfread already installed


In [17]:
# Try reading different file types
import pyreadstat
from dbfread import DBF
import warnings
warnings.filterwarnings('ignore')

file_schemas = {}

# 1. DBF files (zonal_register_data)
print("="*80)
print("📊 DBF FILES (zonal_register_data)")
print("="*80)

dbf_files = [f for f in all_files['zonal_register_data'] if f['extension'] == '.dbf']
print(f"\nTotal DBF files: {len(dbf_files)}")

for dbf_file in dbf_files:
    key = dbf_file['relative_path']
    sample_path = dbf_file['full_path']
    
    try:
        table = DBF(sample_path, encoding='latin-1')
        columns_detail = [
            {
                'name': field.name,
                'type': field.type,
                'length': field.length,
                'decimals': field.decimal_count
            }
            for field in table.fields
        ]
        
        rows = len(table)
        file_schemas[key] = {
            'file_type': 'DBF',
            'rows': rows,
            'columns': len(columns_detail),
            'columns_detail': columns_detail
        }
    except Exception as e:
        file_schemas[key] = {
            'file_type': 'DBF',
            'error': str(e)
        }

print("✓ DBF schema extraction complete")

📊 DBF FILES (zonal_register_data)

Total DBF files: 26
✓ DBF schema extraction complete


In [18]:
# 2. SPSS/SAV files (traveling_survey)
print("\n" + "="*80)
print("📊 SPSS SAV FILES (traveling_survey)")
print("="*80)

sav_files = [f for f in all_files['traveling_survey'] if f['extension'].lower() == '.sav']
print(f"\nTotal SAV files: {len(sav_files)}")

for sav_file in sav_files:
    key = sav_file['relative_path']
    try:
        try:
            df, meta = pyreadstat.read_sav(sav_file['full_path'])
            rows = meta.number_rows if hasattr(meta, 'number_rows') else len(df)
        except TypeError:
            df, meta = pyreadstat.read_sav(sav_file['full_path'])
            rows = len(df)
        
        columns_detail = [
            {'name': col, 'type': str(df[col].dtype)}
            for col in df.columns
        ]
        
        file_schemas[key] = {
            'file_type': 'SAV (SPSS)',
            'rows': rows,
            'columns': len(df.columns),
            'columns_detail': columns_detail
        }
    except Exception as e:
        file_schemas[key] = {
            'file_type': 'SAV (SPSS)',
            'error': str(e)
        }

print("✓ SAV schema extraction complete")


📊 SPSS SAV FILES (traveling_survey)

Total SAV files: 4
✓ SAV schema extraction complete


In [19]:
# 3. XLSX files
print("\n" + "="*80)
print("📊 EXCEL FILES (.xlsx)")
print("="*80)

from openpyxl import load_workbook

xlsx_files = [f for f in all_files['zonal_register_data'] if f['extension'].lower() == '.xlsx']
print(f"\nTotal XLSX files: {len(xlsx_files)}")

for xlsx_file in xlsx_files:
    key = xlsx_file['relative_path']
    try:
        wb = load_workbook(xlsx_file['full_path'], read_only=True, data_only=True)
        sheets_info = []
        
        for sheet_name in wb.sheetnames:
            ws = wb[sheet_name]
            header_row = next(ws.iter_rows(min_row=1, max_row=1, values_only=True), [])
            header = [str(h) if h is not None else '' for h in header_row]
            
            sheets_info.append({
                'sheet': sheet_name,
                'rows': ws.max_row,
                'columns': len(header),
                'column_names': header
            })
        
        file_schemas[key] = {
            'file_type': 'XLSX',
            'sheets': sheets_info
        }
    except Exception as e:
        file_schemas[key] = {
            'file_type': 'XLSX',
            'error': str(e)
        }

print("✓ XLSX schema extraction complete")


📊 EXCEL FILES (.xlsx)

Total XLSX files: 2
✓ XLSX schema extraction complete


In [20]:
# 4. ZIP and other archives
print("\n" + "="*80)
print("📦 ARCHIVES")
print("="*80)

zip_files = [f for f in all_files['traveling_survey'] if f['extension'].lower() == '.zip']
print(f"\nTotal ZIP files: {len(zip_files)}")

import zipfile
for zip_file in zip_files:
    print(f"\n🔍 {zip_file['filename']} ({zip_file['size_mb']:.2f} MB)")
    try:
        with zipfile.ZipFile(zip_file['full_path'], 'r') as z:
            file_list = z.namelist()
            print(f"  Contains {len(file_list)} files:")
            for fname in sorted(file_list)[:10]:  # First 10
                info = z.getinfo(fname)
                size_mb = info.file_size / (1024*1024)
                print(f"    - {fname:<50} {size_mb:>8.2f} MB")
            if len(file_list) > 10:
                print(f"    ... and {len(file_list) - 10} more files")
    except Exception as e:
        print(f"  ⚠️ Could not read: {e}")


📦 ARCHIVES

Total ZIP files: 2

🔍 Filemail.com - Nasjonal RVU akkumulert data.zip (228.44 MB)
  Contains 2 files:
    - RVU 2019-2024 Personfil Vektet 251125.sav             93.56 MB
    - RVU 2019_2024 Reisefil 251107.sav                    134.88 MB

🔍 Filemail.com - RVU 2025.zip (128.01 MB)
  Contains 2 files:
    - Nasjonal_RVU_PERSON_Nov26_0901.sav                    63.65 MB
    - Nasjonal_RVU_REISER_Nov26_0901.sav                    64.36 MB


In [21]:
import numpy as np

print("="*80)
print("📊 DATA PROFILING - Analyzing column values")
print("="*80)

def profile_column(series):
    """Profile a single column (numeric or categorical)"""
    profile = {}
    
    # Basic info
    total_count = len(series)
    null_count = series.isna().sum()
    profile['null_count'] = int(null_count)
    profile['null_pct'] = round(100 * null_count / total_count, 1) if total_count > 0 else 0
    
    # Skip if all nulls
    if null_count == total_count:
        profile['type'] = 'empty'
        return profile
    
    # Check if numeric
    if pd.api.types.is_numeric_dtype(series):
        profile['type'] = 'numeric'
        clean = series.dropna()
        profile['min'] = float(clean.min())
        profile['max'] = float(clean.max())
        profile['mean'] = round(float(clean.mean()), 2)
        profile['median'] = float(clean.median())
        profile['unique'] = int(series.nunique())
    else:
        # Categorical
        profile['type'] = 'categorical'
        profile['unique'] = int(series.nunique())
        
        # Top 5 values with counts
        value_counts = series.value_counts().head(5)
        profile['top_values'] = [
            {'value': str(val), 'count': int(count), 'pct': round(100 * count / total_count, 1)}
            for val, count in value_counts.items()
        ]
    
    return profile

# Profile DBF files
print("\n1. Profiling DBF files...")
dbf_files = [f for f in all_files['zonal_register_data'] if f['extension'] == '.dbf']

for idx, dbf_file in enumerate(dbf_files):
    key = dbf_file['relative_path']
    
    if key not in file_schemas or 'error' in file_schemas[key]:
        continue
    
    print(f"  [{idx+1}/{len(dbf_files)}] {dbf_file['filename'][:50]}...", end=" ")
    
    try:
        # Read full file
        table = DBF(dbf_file['full_path'], encoding='latin-1')
        df = pd.DataFrame(list(table))
        
        # Profile each column
        column_profiles = {}
        for col in df.columns:
            column_profiles[col] = profile_column(df[col])
        
        file_schemas[key]['column_profiles'] = column_profiles
        print("✓")
    except Exception as e:
        print(f"✗ Error: {str(e)[:50]}")
        file_schemas[key]['profile_error'] = str(e)

# Profile SAV files
print("\n2. Profiling SAV files...")
sav_files = [f for f in all_files['traveling_survey'] if f['extension'].lower() == '.sav']

for idx, sav_file in enumerate(sav_files):
    key = sav_file['relative_path']
    
    if key not in file_schemas or 'error' in file_schemas[key]:
        continue
    
    print(f"  [{idx+1}/{len(sav_files)}] {sav_file['filename'][:50]}...", end=" ")
    
    try:
        # Read full file
        df, meta = pyreadstat.read_sav(sav_file['full_path'])
        
        # Profile each column
        column_profiles = {}
        for col in df.columns:
            column_profiles[col] = profile_column(df[col])
        
        file_schemas[key]['column_profiles'] = column_profiles
        print("✓")
    except Exception as e:
        print(f"✗ Error: {str(e)[:50]}")
        file_schemas[key]['profile_error'] = str(e)

print("\n✅ Data profiling complete!")
print(f"   DBF files profiled: {sum(1 for f in dbf_files if 'column_profiles' in file_schemas.get(f['relative_path'], {}))}/{len(dbf_files)}")
print(f"   SAV files profiled: {sum(1 for f in sav_files if 'column_profiles' in file_schemas.get(f['relative_path'], {}))}/{len(sav_files)}")

📊 DATA PROFILING - Analyzing column values

1. Profiling DBF files...
  [1/26] sdat1_d2024_g2020.dbf... ✓
  [2/26] sdat1_d2024_g2021.dbf... ✓
  [3/26] sdat1_d2024_g2023.dbf... ✓
  [4/26] sdat1_d2024_g2024.dbf... ✓
  [5/26] sdat3_d2023x_g2020.dbf... ✓
  [6/26] sdat3_d2023x_g2021.dbf... ✓
  [7/26] sdat3_d2023x_g2023.dbf... ✓
  [8/26] sdat3_d2023x_g2024.dbf... ✓
  [9/26] sdat4_d2024_g2020.dbf... ✓
  [10/26] sdat4_d2024_g2021.dbf... ✓
  [11/26] sdat4_d2024_g2023.dbf... ✓
  [12/26] sdat4_d2024_g2024.dbf... ✓
  [13/26] sdat5_d2023_g2020.dbf... ✓
  [14/26] sdat5_d2023_g2021.dbf... ✓
  [15/26] sdat5_d2023_g2023.dbf... ✓
  [16/26] sdat5_d2023_g2024.dbf... ✓
  [17/26] sdat71_NB2023_grk2020_2020.dbf... ✓
  [18/26] sdat7_d20xx_g2020_ikke_pkost.dbf... ✓
  [19/26] sdat7_d20xx_g2021_ikke_pkost.dbf... ✓
  [20/26] sdat7_d20xx_g2023_ikke_pkost.dbf... ✓
  [21/26] sdat7_d20xx_g2024_ikke_pkost.dbf... ✓
  [22/26] sdat8_d2024_g2020.dbf... ✓
  [23/26] sdat8_d2024_g2021.dbf... ✓
  [24/26] sdat8_d2024_g2023.dbf

## 2.5 Data Profiling

Profile each column to extract:
- **Numeric columns:** min, max, mean, median
- **Categorical columns:** unique count, top 5 most frequent values

## 3. Generate Data Catalog Markdown

Create a comprehensive markdown document of all datasets.

In [31]:
# Generate comprehensive markdown catalog
import re

catalog_md = f"""# RVX Data Catalog

**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

Complete inventory and documentation of all datasets in the RVX folders.

## Overview

The RVX folder contains two main data sources:

1. **traveling_survey/** - National travel survey (RVU - Reisevaneundersøkelsen)
2. **zonal_register_data/** - Zonal statistical data (SDAT files) from Statistics Norway

## Folder Structure

"""

def group_sdat_files(files):
    sdat_files = [
        f for f in files
        if f['filename'].lower().startswith('sdat') and f['extension'] == '.dbf'
    ]
    other_files = [f for f in files if f not in sdat_files]

    groups = {}
    for f in sdat_files:
        name = f['filename'].lower()
        match = re.match(r"^(sdat\d+)_", name)
        if not match:
            match = re.match(r"^(sdat_\d+)_", name)
        if match:
            key = match.group(1)
        else:
            key = name.split('.')[0]
        groups.setdefault(key, []).append(f)

    return groups, other_files

def group_xlsx_files(files):
    """Group XLSX files by prefix (e.g., sdat2_data2020_*)"""
    xlsx_files = [
        f for f in files
        if f['extension'].lower() == '.xlsx'
    ]
    other_files = [f for f in files if f not in xlsx_files]
    
    groups = {}
    for f in xlsx_files:
        name = f['filename'].lower()
        # Extract prefix like "sdat2_data2020"
        match = re.match(r"^(sdat\d+_data\d+)_", name)
        if match:
            key = match.group(1)
        else:
            key = name.split('_')[0] if '_' in name else name.split('.')[0]
        groups.setdefault(key, []).append(f)
    
    return groups, other_files

def get_demographic_breakdown():
    """Returns documentation about the demographic structure in XLSX files"""
    return """
**Column Structure & Demographics:**

The 360+ columns in each geographic area are structured as:
**24 demographic cells** (age × gender) × **15 household categories** = **360 variables**

**Age & Gender (24 cells):**
- 12 age groups × 2 genders = 24 combinations

**Household Categories (15 categories):**
- Number of adults: 1 adult / 2 adults / 3+ adults
- Household type:
  - Enslig uten barn (Single without children)
  - Enslig med barn (Single with children)
  - Par uten barn (Couple without children)
  - Par med barn (Couple with children)
  - Flere voksne (Multiple adults)

**Calculation:** 3 adult categories × 5 household types = 15 categories per age/gender group
"""

def summarize_group(group_files):
    sizes = sum(f['size_mb'] for f in group_files)
    schemas = [file_schemas.get(f['relative_path'], {}) for f in group_files]

    rows_list = [s.get('rows') for s in schemas if isinstance(s.get('rows'), int)]
    cols_list = [s.get('columns') for s in schemas if isinstance(s.get('columns'), int)]

    if rows_list:
        rows = f"{min(rows_list)}–{max(rows_list)}" if min(rows_list) != max(rows_list) else f"{rows_list[0]}"
    else:
        rows = "n/a"

    if cols_list:
        cols = f"{cols_list[0]}" if len(set(cols_list)) == 1 else f"{min(cols_list)}–{max(cols_list)}"
    else:
        cols = "n/a"

    return sizes, rows, cols

# Add folder summaries
for folder_name in folders:
    files = all_files[folder_name]
    total_size = sum(f['size_mb'] for f in files)

    catalog_md += f"\n### {folder_name}/\n\n"

    if folder_name == 'traveling_survey':
        catalog_md += """**National Travel Survey Data (RVU)**

Contains survey responses about travel behavior of Norwegian households.
- **Format:** SPSS (.sav), ZIP archives, documentation (.docx)
- **Source:** Statistics Norway (SSB)
- **Coverage:** Years 2019-2025

"""
    else:
        catalog_md += """**Zonal Statistical Data (SDAT)**

Grid-based statistical data at different geographic resolutions.
- **Format:** DBF (dBase), XLSX
- **Source:** Statistics Norway (TRAMOD/RVX)
- **Coverage:** Multiple grid resolutions (grunnkrets, delomr, etc.)
- **Data years:** 2020-2024

"""

    catalog_md += f"**Statistics:**\n"
    catalog_md += f"- Total files: {len(files)}\n"
    catalog_md += f"- Total size: {total_size:.2f} MB\n\n"

    # List files
    catalog_md += "**Files:**\n\n"
    catalog_md += "| Filename | Size (MB) | Type | Rows | Columns |\n"
    catalog_md += "|----------|-----------|------|------|---------|\n"

    if folder_name == 'zonal_register_data':
        groups, other_files = group_sdat_files(files)

        for group_key in sorted(groups.keys()):
            group_files = sorted(groups[group_key], key=lambda x: x['filename'])
            size_mb, rows, cols = summarize_group(group_files)
            label = f"{group_key}_* ({len(group_files)} files)"
            catalog_md += f"| `{label}` | {size_mb:.2f} | .dbf | {rows} | {cols} |\n"

        for f in sorted(other_files, key=lambda x: x['filename']):
            schema = file_schemas.get(f['relative_path'], {})
            rows = schema.get('rows', 'n/a')
            cols = schema.get('columns', 'n/a')
            catalog_md += f"| `{f['relative_path']}` | {f['size_mb']:.2f} | {f['extension'] or 'dir'} | {rows} | {cols} |\n"
    else:
        for f in files:
            schema = file_schemas.get(f['relative_path'], {})
            rows = schema.get('rows', 'n/a')
            cols = schema.get('columns', 'n/a')
            catalog_md += f"| `{f['relative_path']}` | {f['size_mb']:.2f} | {f['extension'] or 'dir'} | {rows} | {cols} |\n"

    catalog_md += "\n"

# Add detailed schema section
catalog_md += "## Dataset Schemas\n\n"

for folder_name in folders:
    catalog_md += f"### {folder_name}/\n\n"
    files = all_files[folder_name]

    if folder_name == 'zonal_register_data':
        groups, other_files = group_sdat_files(files)

        for group_key in sorted(groups.keys()):
            group_files = sorted(groups[group_key], key=lambda x: x['filename'])
            base_file = group_files[0]
            base_schema = file_schemas.get(base_file['relative_path'], {})

            cols_detail = base_schema.get('columns_detail', [])
            base_cols = [c.get('name', '') for c in cols_detail]

            all_same = True
            for f in group_files[1:]:
                schema = file_schemas.get(f['relative_path'], {})
                cols = [c.get('name', '') for c in schema.get('columns_detail', [])]
                if cols != base_cols:
                    all_same = False
                    break

            size_mb, rows, cols = summarize_group(group_files)
            file_list = ", ".join([f"{f['filename']}" for f in group_files])

            catalog_md += f"#### {group_key}_* ({len(group_files)} files)\n\n"
            catalog_md += f"- **Files:** {file_list}\n"
            catalog_md += f"- **Type:** DBF\n"
            catalog_md += f"- **Rows:** {rows}\n"
            catalog_md += f"- **Columns:** {cols}\n\n"

            if all_same and cols_detail:
                columns_detail = base_schema.get('columns_detail', [])
                column_profiles = base_schema.get('column_profiles', {})
                
                catalog_md += "**Column details (shared across group):**\n\n"
                if column_profiles:
                    catalog_md += "| Column | Type | Range/Values | Nulls |\n"
                    catalog_md += "|--------|------|--------------|-------|\n"
                    for col in cols_detail:
                        col_name = col.get('name', '')
                        col_type = col.get('type', '')
                        profile = column_profiles.get(col_name, {})
                        
                        # Format range/values
                        if profile.get('type') == 'numeric':
                            range_str = f"{profile.get('min', '?')} – {profile.get('max', '?')} (μ={profile.get('mean', '?')})"
                        elif profile.get('type') == 'categorical':
                            top = profile.get('top_values', [])[:3]
                            if top:
                                range_str = ', '.join([f"{v['value']}({v['pct']}%)" for v in top])
                                if profile.get('unique', 0) > 3:
                                    range_str += f" ... ({profile['unique']} unique)"
                            else:
                                range_str = f"{profile.get('unique', '?')} unique"
                        else:
                            range_str = "—"
                        
                        # Format nulls
                        null_pct = profile.get('null_pct', 0)
                        null_str = f"{null_pct}%" if null_pct > 0 else "—"
                        
                        catalog_md += f"| {col_name} | {col_type} | {range_str} | {null_str} |\n"
                else:
                    catalog_md += "| Column | Type |\n"
                    catalog_md += "|--------|------|\n"
                    for col in cols_detail:
                        col_name = col.get('name', '')
                        col_type = col.get('type', '')
                        catalog_md += f"| {col_name} | {col_type} |\n"
                catalog_md += "\n"
            else:
                catalog_md += "**Note:** Columns differ across files in this group.\n\n"

        # Non-SDAT files - group XLSX files by prefix
        xlsx_groups, remaining_files = group_xlsx_files(other_files)
        
        for group_key in sorted(xlsx_groups.keys()):
            group_files = sorted(xlsx_groups[group_key], key=lambda x: x['filename'])
            
            # Get columns from each file
            group_schemas = {}
            for f in group_files:
                key = f['relative_path']
                schema = file_schemas.get(key, {})
                sheets = schema.get('sheets', [])
                if sheets:
                    col_names = sheets[0].get('column_names', [])
                    group_schemas[f['filename']] = col_names
            
            # Check if all files have identical columns
            base_file = group_files[0]
            base_cols = group_schemas.get(base_file['filename'], [])
            
            all_same = True
            different_cols = {}
            for f in group_files[1:]:
                cols = group_schemas.get(f['filename'], [])
                if cols != base_cols:
                    all_same = False
                    missing = [c for c in base_cols if c not in cols]
                    extra = [c for c in cols if c not in base_cols]
                    different_cols[f['filename']] = {'missing': missing, 'extra': extra}
            
            file_list = ", ".join([f"{f['filename']}" for f in group_files])
            base_schema = file_schemas.get(base_file['relative_path'], {})
            sheets = base_schema.get('sheets', [])
            
            catalog_md += f"#### {group_key}_* ({len(group_files)} files)\n\n"
            catalog_md += f"- **Files:** {file_list}\n"
            catalog_md += f"- **Type:** XLSX\n"
            
            if sheets:
                catalog_md += f"- **Sheets per file:** {len(sheets)}\n"
                if all_same:
                    catalog_md += f"- **Common columns:** {len(base_cols)}\n"
                    catalog_md += f"- **Shared schema:** ✅ All files have identical column structure\n\n"
                else:
                    catalog_md += f"- **Shared columns:** {len(base_cols)}\n"
                    catalog_md += f"- **Schema:** Geographic-level variations\n\n"
                
                # Add demographic breakdown explanation
                catalog_md += get_demographic_breakdown()
                catalog_md += "\n\n"
                
                # Show common columns
                catalog_md += "**Common columns (subset - first 10):**\n\n"
                catalog_md += "| Column | Description |\n"
                catalog_md += "|--------|-------------|\n"
                for col in base_cols[:10]:
                    catalog_md += f"| {col} | Demographic × Household |\n"
                if len(base_cols) > 10:
                    catalog_md += f"| ... | ({len(base_cols) - 10} more demographic combinations) |\n"
                catalog_md += "\n"
                
                # If not all same, show differences
                if not all_same:
                    catalog_md += "**Geographic-level differences:**\n\n"
                    for fname, diffs in different_cols.items():
                        if diffs['extra']:
                            catalog_md += f"- **{fname}**: +{', '.join(diffs['extra'])}\n"
                    catalog_md += "\n"
            catalog_md += "\n"
        
        # Handle remaining non-grouped files
        for f in sorted(remaining_files, key=lambda x: x['filename']):
            key = f['relative_path']
            schema = file_schemas.get(key)
            if not schema:
                continue

            catalog_md += f"#### {f['filename']}\n\n"
            catalog_md += f"- **Path:** `{key}`\n"
            catalog_md += f"- **Type:** {schema.get('file_type', 'Unknown')}\n"

            if 'error' in schema:
                catalog_md += f"- **Error:** {schema['error']}\n\n"
                continue
    else:
        for f in files:
            key = f['relative_path']
            schema = file_schemas.get(key)
            if not schema:
                continue

            catalog_md += f"#### {f['filename']}\n\n"
            catalog_md += f"- **Path:** `{key}`\n"
            catalog_md += f"- **Type:** {schema.get('file_type', 'Unknown')}\n"

            if 'error' in schema:
                catalog_md += f"- **Error:** {schema['error']}\n\n"
                continue

            if schema.get('file_type') == 'XLSX':
                catalog_md += f"- **Sheets:** {len(schema.get('sheets', []))}\n\n"
                for sheet in schema.get('sheets', []):
                    catalog_md += f"  - **Sheet:** {sheet['sheet']}\n"
                    catalog_md += f"    - Rows: {sheet['rows']}\n"
                    catalog_md += f"    - Columns: {sheet['columns']}\n"
                    catalog_md += f"    - Column names: {', '.join(sheet['column_names'])}\n"
                catalog_md += "\n"
            else:
                catalog_md += f"- **Rows:** {schema.get('rows', 'n/a')}\n"
                catalog_md += f"- **Columns:** {schema.get('columns', 'n/a')}\n\n"

                columns_detail = schema.get('columns_detail', [])
                column_profiles = schema.get('column_profiles', {})
                
                if columns_detail:
                    catalog_md += "**Column details:**\n\n"
                    if column_profiles:
                        catalog_md += "| Column | Type | Range/Values | Nulls |\n"
                        catalog_md += "|--------|------|--------------|-------|\n"
                        for col in columns_detail:
                            col_name = col.get('name', '')
                            col_type = col.get('type', '')
                            profile = column_profiles.get(col_name, {})
                            
                            # Format range/values
                            if profile.get('type') == 'numeric':
                                range_str = f"{profile.get('min', '?')} – {profile.get('max', '?')} (μ={profile.get('mean', '?')})"
                            elif profile.get('type') == 'categorical':
                                top = profile.get('top_values', [])[:3]
                                if top:
                                    range_str = ', '.join([f"{v['value']}({v['pct']}%)" for v in top])
                                    if profile.get('unique', 0) > 3:
                                        range_str += f" ... ({profile['unique']} unique)"
                                else:
                                    range_str = f"{profile.get('unique', '?')} unique"
                            else:
                                range_str = "—"
                            
                            # Format nulls
                            null_pct = profile.get('null_pct', 0)
                            null_str = f"{null_pct}%" if null_pct > 0 else "—"
                            
                            catalog_md += f"| {col_name} | {col_type} | {range_str} | {null_str} |\n"
                    else:
                        catalog_md += "| Column | Type |\n"
                        catalog_md += "|--------|------|\n"
                        for col in columns_detail:
                            col_name = col.get('name', '')
                            col_type = col.get('type', '')
                            catalog_md += f"| {col_name} | {col_type} |\n"
                    catalog_md += "\n"

print("Generated catalog (first 2000 chars):")
print(catalog_md[:2000])

Generated catalog (first 2000 chars):
# RVX Data Catalog

**Generated:** 2026-02-04 11:58:18

Complete inventory and documentation of all datasets in the RVX folders.

## Overview

The RVX folder contains two main data sources:

1. **traveling_survey/** - National travel survey (RVU - Reisevaneundersøkelsen)
2. **zonal_register_data/** - Zonal statistical data (SDAT files) from Statistics Norway

## Folder Structure


### traveling_survey/

**National Travel Survey Data (RVU)**

Contains survey responses about travel behavior of Norwegian households.
- **Format:** SPSS (.sav), ZIP archives, documentation (.docx)
- **Source:** Statistics Norway (SSB)
- **Coverage:** Years 2019-2025

**Statistics:**
- Total files: 8
- Total size: 713.47 MB

**Files:**

| Filename | Size (MB) | Type | Rows | Columns |
|----------|-----------|------|------|---------|
| `Filemail.com - Nasjonal RVU akkumulert data.zip` | 228.44 | .zip | n/a | n/a |
| `Filemail.com - RVU 2025.zip` | 128.01 | .zip | n/a | n/a

In [32]:
# Save the catalog markdown
catalog_path = project_root / 'docs' / 'DATA_CATALOG_RVX.md'
catalog_path.parent.mkdir(parents=True, exist_ok=True)

with open(catalog_path, 'w') as f:
    f.write(catalog_md)

print(f"✓ Saved catalog to: {catalog_path}")
print(f"\nFile size: {catalog_path.stat().st_size / 1024:.2f} KB")

✓ Saved catalog to: /Users/anderskielland/Documents/Synthetic data/code/synthetic-lab/docs/DATA_CATALOG_RVX.md

File size: 67.65 KB


## 4. Summary

Check what we've discovered

In [24]:
print("\n" + "="*80)
print("DATA CATALOG SUMMARY")
print("="*80)

for folder_name in folders:
    files = all_files[folder_name]
    total_size = sum(f['size_mb'] for f in files)
    
    print(f"\n📁 {folder_name}:")
    print(f"   Files: {len(files)}")
    print(f"   Total size: {total_size:.2f} MB")
    
    by_ext = defaultdict(int)
    for f in files:
        by_ext[f['extension']] += 1
    
    # Build type summary string
    type_summary = ', '.join(f"{ext or '[none]'}({c})" for ext, c in sorted(by_ext.items()))
    print(f"   Types: {type_summary}")

print(f"\n✅ Full catalog saved to: {catalog_path}")


DATA CATALOG SUMMARY

📁 traveling_survey:
   Files: 8
   Total size: 713.47 MB
   Types: .docx(2), .sav(4), .zip(2)

📁 zonal_register_data:
   Files: 28
   Total size: 79.42 MB
   Types: .dbf(26), .xlsx(2)

✅ Full catalog saved to: /Users/anderskielland/Documents/Synthetic data/code/synthetic-lab/docs/DATA_CATALOG_RVX.md


## 5. Create Pretty Outputs
Convert the markdown catalog to HTML 

In [33]:
# 2. Create styled HTML version (beautiful webpage)
# Convert markdown to HTML first
import markdown
html_content = markdown.markdown(catalog_md, extensions=['tables'])

# Create full HTML page with styling (using f-string to avoid format conflicts)
full_html = f"""<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>RVX Data Catalog</title>
    <style>
        body {{
            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, Oxygen, Ubuntu, sans-serif;
            line-height: 1.6;
            max-width: 1200px;
            margin: 0 auto;
            padding: 20px;
            background-color: #f5f5f5;
        }}
        .container {{
            background-color: white;
            padding: 40px;
            border-radius: 8px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
        }}
        h1 {{
            color: #2c3e50;
            border-bottom: 3px solid #3498db;
            padding-bottom: 10px;
        }}
        h2 {{
            color: #34495e;
            margin-top: 30px;
            border-bottom: 2px solid #ecf0f1;
            padding-bottom: 8px;
        }}
        h3 {{
            color: #7f8c8d;
        }}
        table {{
            width: 100%;
            border-collapse: collapse;
            margin: 20px 0;
            background-color: white;
        }}
        th {{
            background-color: #3498db;
            color: white;
            padding: 12px;
            text-align: left;
            font-weight: 600;
        }}
        td {{
            padding: 10px 12px;
            border-bottom: 1px solid #ecf0f1;
        }}
        tr:hover {{
            background-color: #f8f9fa;
        }}
        code {{
            background-color: #f4f4f4;
            padding: 2px 6px;
            border-radius: 3px;
            font-family: 'Courier New', monospace;
            font-size: 0.9em;
        }}
        ul {{
            padding-left: 25px;
        }}
        li {{
            margin: 8px 0;
        }}
        .generated-date {{
            color: #95a5a6;
            font-style: italic;
        }}
    </style>
</head>
<body>
    <div class="container">
        {html_content}
    </div>
</body>
</html>
"""

# Save HTML file
html_path = project_root / 'docs' / 'DATA_CATALOG_RVX.html'
with open(html_path, 'w', encoding='utf-8') as f:
    f.write(full_html)

print(f"✅ Saved styled HTML to: {html_path}")
print(f"   Open in browser: file://{html_path}")
print(f"   File size: {html_path.stat().st_size / 1024:.2f} KB")

✅ Saved styled HTML to: /Users/anderskielland/Documents/Synthetic data/code/synthetic-lab/docs/DATA_CATALOG_RVX.html
   Open in browser: file:///Users/anderskielland/Documents/Synthetic data/code/synthetic-lab/docs/DATA_CATALOG_RVX.html
   File size: 106.20 KB


## 6. Interactive Data Inspector

Browse through DBF files to verify schema and content.

In [33]:
# List all available DBF files
print("="*80)
print("📋 AVAILABLE DBF FILES")
print("="*80)
print("\nChange the file_index variable below to inspect different files:\n")

dbf_file_list = [f for f in all_files['zonal_register_data'] if f['extension'] == '.dbf']

for idx, f in enumerate(dbf_file_list):
    print(f"{idx:2d}. {f['filename']:<50} ({f['size_mb']:.2f} MB, {file_schemas.get(f['relative_path'], {}).get('rows', '?')} rows)")

print(f"\nTotal: {len(dbf_file_list)} DBF files")

📋 AVAILABLE DBF FILES

Change the file_index variable below to inspect different files:

 0. sdat1_d2024_g2020.dbf                              (4.42 MB, 14097 rows)
 1. sdat1_d2024_g2021.dbf                              (4.42 MB, 14097 rows)
 2. sdat1_d2024_g2023.dbf                              (4.43 MB, 14101 rows)
 3. sdat1_d2024_g2024.dbf                              (4.43 MB, 14101 rows)
 4. sdat3_d2023x_g2020.dbf                             (2.38 MB, 14097 rows)
 5. sdat3_d2023x_g2021.dbf                             (2.38 MB, 14097 rows)
 6. sdat3_d2023x_g2023.dbf                             (2.38 MB, 14101 rows)
 7. sdat3_d2023x_g2024.dbf                             (2.38 MB, 14101 rows)
 8. sdat4_d2024_g2020.dbf                              (2.60 MB, 14097 rows)
 9. sdat4_d2024_g2021.dbf                              (2.60 MB, 14097 rows)
10. sdat4_d2024_g2023.dbf                              (2.60 MB, 14101 rows)
11. sdat4_d2024_g2024.dbf                              (2.60 MB,

In [59]:
# Change this number to inspect different files (0 to 25)
file_index = 9

# Load and display
selected = dbf_file_list[file_index]
print(selected)
table = DBF(selected['full_path'], encoding='latin-1')
df = pd.DataFrame(list(table))

print(f"{selected['filename']} ({selected['size_mb']:.2f} MB)\n")
print(f"{df.shape[0]} rows x {df.shape[1]} columns\n")

# Generic schema check: column names and DBF types (2-row table)
row_names = [field.name for field in table.fields]
row_types = [field.type for field in table.fields]

schema_check = pd.DataFrame([row_names, row_types], index=["Column", "DBF Type"])
print("Schema check (columns + DBF type):")
display(schema_check)

# Raw preview
df.head(6)

{'filename': 'sdat4_d2024_g2021.dbf', 'relative_path': 'sdat4_d2024_g2021.dbf', 'full_path': '/Users/anderskielland/Documents/Synthetic data/code/synthetic-lab/data/raw/population/rvx/zonal_register_data/sdat4_d2024_g2021.dbf', 'size_bytes': 2721523, 'size_mb': 2.6, 'extension': '.dbf'}
sdat4_d2024_g2021.dbf (2.60 MB)

14097 rows x 24 columns

Schema check (columns + DBF type):


,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
Column,GRUNNKRETS,A10PRI,A20SEK,A21SEK,A30VH,A31VH,A32VH,A33VH,A34VH,A40TJE,...,A60UND,A61UND,A62UND,A63UND,A70HSOS,A71HSOS,A72HSOS,A73HSOS,MALINT,FEMINT
DBF Type,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N


,GRUNNKRETS,A10PRI,A20SEK,A21SEK,A30VH,A31VH,A32VH,A33VH,A34VH,A40TJE,...,A60UND,A61UND,A62UND,A63UND,A70HSOS,A71HSOS,A72HSOS,A73HSOS,MALINT,FEMINT
0,3010101,1.0,165.0,0.0,123.0,4.0,34.0,315.0,0.0,1060.0,...,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,747.0,5.0
1,3010102,0.0,545.0,0.0,238.0,73.0,324.0,489.0,5.0,9090.0,...,122.0,0.0,44.0,33.0,465.0,48.0,6.0,0.0,3243.0,197.0
2,3010103,0.0,372.0,0.0,407.0,112.0,1324.0,1023.0,0.0,6369.0,...,44.0,0.0,0.0,0.0,30.0,58.0,0.0,0.0,1625.0,956.0
3,3010104,0.0,1510.0,0.0,344.0,434.0,880.0,1494.0,1.0,5046.0,...,52.0,0.0,0.0,1521.0,78.0,24.0,0.0,1.0,542.0,693.0
4,3010105,30.0,467.0,0.0,161.0,25.0,36.0,447.0,6.0,4881.0,...,8.0,0.0,0.0,5.0,1321.0,41.0,3.0,32.0,1220.0,153.0
5,3010201,0.0,2254.0,0.0,340.0,40.0,65.0,111.0,0.0,4302.0,...,104.0,0.0,182.0,0.0,495.0,25.0,1.0,7.0,1693.0,311.0


In [60]:
dbf_file_list

[{'filename': 'sdat1_d2024_g2020.dbf',
  'relative_path': 'sdat1_d2024_g2020.dbf',
  'full_path': '/Users/anderskielland/Documents/Synthetic data/code/synthetic-lab/data/raw/population/rvx/zonal_register_data/sdat1_d2024_g2020.dbf',
  'size_bytes': 4639259,
  'size_mb': 4.42,
  'extension': '.dbf'},
 {'filename': 'sdat1_d2024_g2021.dbf',
  'relative_path': 'sdat1_d2024_g2021.dbf',
  'full_path': '/Users/anderskielland/Documents/Synthetic data/code/synthetic-lab/data/raw/population/rvx/zonal_register_data/sdat1_d2024_g2021.dbf',
  'size_bytes': 4639259,
  'size_mb': 4.42,
  'extension': '.dbf'},
 {'filename': 'sdat1_d2024_g2023.dbf',
  'relative_path': 'sdat1_d2024_g2023.dbf',
  'full_path': '/Users/anderskielland/Documents/Synthetic data/code/synthetic-lab/data/raw/population/rvx/zonal_register_data/sdat1_d2024_g2023.dbf',
  'size_bytes': 4640575,
  'size_mb': 4.43,
  'extension': '.dbf'},
 {'filename': 'sdat1_d2024_g2024.dbf',
  'relative_path': 'sdat1_d2024_g2024.dbf',
  'full_path':

## 7. Interactive Survey Data Inspector

Browse through SAV (SPSS) files from the traveling_survey folder.

In [61]:
# List all available SAV files
print("="*80)
print("📋 AVAILABLE SAV FILES (Survey Data)")
print("="*80)
print("\nChange the sav_file_index variable below to inspect different files:\n")

sav_file_list = [f for f in all_files['traveling_survey'] if f['extension'].lower() == '.sav']

for idx, f in enumerate(sav_file_list):
    print(f"{idx:2d}. {f['filename']:<50} ({f['size_mb']:.2f} MB, {file_schemas.get(f['relative_path'], {}).get('rows', '?')} rows)")

print(f"\nTotal: {len(sav_file_list)} SAV files")
print("\nNote: .docx files are documentation, .zip files are compressed archives")

📋 AVAILABLE SAV FILES (Survey Data)

Change the sav_file_index variable below to inspect different files:

 0. Nasjonal_RVU_PERSON_Nov26_0901.sav                 (63.65 MB, 100 rows)
 1. Nasjonal_RVU_REISER_Nov26_0901.sav                 (64.36 MB, 100 rows)
 2. RVU 2019-2024 Personfil Vektet 251125.sav          (93.56 MB, ? rows)
 3. RVU 2019_2024 Reisefil 251107.sav                  (134.88 MB, 100 rows)

Total: 4 SAV files

Note: .docx files are documentation, .zip files are compressed archives


In [72]:
# Change this number to inspect different files (0 to 3)
sav_file_index = 0

# Load and display
selected = sav_file_list[sav_file_index]
df, meta = pyreadstat.read_sav(selected['full_path'])

# Get actual total rows from metadata
total_rows = meta.number_rows if hasattr(meta, 'number_rows') else len(df)

print(f"{selected['filename']} ({selected['size_mb']:.2f} MB)\n")
print(f"{total_rows} rows x {df.shape[1]} columns\n")

# Schema check: column names and types
col_names = list(df.columns)
col_types = [str(df[col].dtype) for col in df.columns]

schema_check = pd.DataFrame([col_names, col_types], index=["Column", "Type"])
print("Schema check (columns + pandas type):")
display(schema_check)

# Raw preview
df.head(6)

Nasjonal_RVU_PERSON_Nov26_0901.sav (63.65 MB)

51330 rows x 453 columns

Schema check (columns + pandas type):


,0,1,2,3,4,5,6,7,8,9,...,443,444,445,446,447,448,449,450,451,452
Column,altid,respid,altid_1,altidNumeric,PublishedVersion,date_time_start_1,date_time_start_2,postnummer,alder,Aldersgruppe,...,TRM_BRUK_PERSON_18,TRM_BRUK_PERSON_20,TRM_BRUK_PERSON_21,TRM_BRUK_PERSON_22,TRM_BRUK_PERSON_23,TRM_BRUK_PERSON_97,TRM_BRUK_PERSON_70,TRM_BRUK_PERSON_71,utvalgsvekt_kombinert_Q1_Q3,utvalgsvekt_nasjonal_Q1_Q3
Type,object,object,object,float64,float64,float64,float64,object,float64,float64,...,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64


,altid,respid,altid_1,altidNumeric,PublishedVersion,date_time_start_1,date_time_start_2,postnummer,alder,Aldersgruppe,...,TRM_BRUK_PERSON_18,TRM_BRUK_PERSON_20,TRM_BRUK_PERSON_21,TRM_BRUK_PERSON_22,TRM_BRUK_PERSON_23,TRM_BRUK_PERSON_97,TRM_BRUK_PERSON_70,TRM_BRUK_PERSON_71,utvalgsvekt_kombinert_Q1_Q3,utvalgsvekt_nasjonal_Q1_Q3
0,2225kj96,529507ea-efa4-4a0a-b39a-381f6ff2d9ae,2225kj96,0.0,19.0,20250505.0,73201.0,1396,37.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.052456,1.052456
1,222ce9b3,3a5f1c25-0275-4804-81b2-1fce3b7514c3,222ce9b3,0.0,101.0,20250521.0,161939.0,5310,40.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.819318,NaN
2,222mygj8,bc00ba46-9824-4328-810c-b18376aeb06b,222mygj8,0.0,18.0,20250411.0,145352.0,7072,17.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.845151,NaN
3,22444jw3,ed53a78e-29dd-4160-a2c7-748d8abc1d23,22444jw3,0.0,103.0,20250606.0,94329.0,9007,15.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.975325,NaN
4,22476dch,914f2d07-2ed1-4c84-806f-8b0d20163108,22476dch,0.0,106.0,20250907.0,93902.0,4640,58.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.433040,NaN
5,224kguwm,bea0d537-7181-4a73-b5bc-d3461205f755,224kguwm,0.0,104.0,20250701.0,230049.0,9108,40.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.923852,NaN


In [75]:
import re

# Check if files with same SDAT number share the same columns
sdat_files = [
    f for f in all_files['zonal_register_data']
    if f['filename'].lower().startswith('sdat') and f['extension'] == '.dbf'
]

# Group by exact prefix like sdat7_ vs sdat71_ (underscore boundary)
groups = {}
for f in sdat_files:
    name = f['filename'].lower()
    match = re.match(r"^(sdat\d+)_", name)
    if not match:
        continue
    key = match.group(1)  # e.g. sdat7, sdat71
    groups.setdefault(key, []).append(f)

print(f"Found {len(sdat_files)} SDAT files across {len(groups)} groups.")

for group_key in sorted(groups.keys()):
    group_files = groups[group_key]
    schemas = {}
    for f in group_files:
        schema = file_schemas.get(f['relative_path'], {})
        cols = [c['name'] for c in schema.get('columns_detail', [])]
        schemas[f['filename']] = cols

    base_file = group_files[0]['filename']
    base_cols = schemas.get(base_file, [])

    print("\n" + "=" * 80)
    print(f"Group: {group_key} ({len(group_files)} files)")
    print(f"Baseline: {base_file} ({len(base_cols)} columns)")

    all_same = True
    for fname, cols in schemas.items():
        if cols != base_cols:
            all_same = False
            missing = [c for c in base_cols if c not in cols]
            extra = [c for c in cols if c not in base_cols]
            print(f"\nDifferences in {fname}:")
            if missing:
                print(f"  Missing: {missing}")
            if extra:
                print(f"  Extra: {extra}")

    if all_same:
        print("✅ All files in this group have identical columns.")

Found 26 SDAT files across 7 groups.

Group: sdat1 (4 files)
Baseline: sdat1_d2024_g2020.dbf (41 columns)
✅ All files in this group have identical columns.

Group: sdat3 (4 files)
Baseline: sdat3_d2023x_g2020.dbf (18 columns)
✅ All files in this group have identical columns.

Group: sdat4 (4 files)
Baseline: sdat4_d2024_g2020.dbf (24 columns)
✅ All files in this group have identical columns.

Group: sdat5 (4 files)
Baseline: sdat5_d2023_g2020.dbf (5 columns)
✅ All files in this group have identical columns.

Group: sdat7 (4 files)
Baseline: sdat7_d20xx_g2020_ikke_pkost.dbf (6 columns)
✅ All files in this group have identical columns.

Group: sdat71 (1 files)
Baseline: sdat71_NB2023_grk2020_2020.dbf (4 columns)
✅ All files in this group have identical columns.

Group: sdat8 (4 files)
Baseline: sdat8_d2024_g2020.dbf (8 columns)
✅ All files in this group have identical columns.


In [27]:
import re

# Check XLSX files - group similar files and compare columns
xlsx_files = [
    f for f in all_files['zonal_register_data']
    if f['extension'].lower() == '.xlsx'
]

print("="*80)
print("📊 XLSX FILE COLUMN COMPARISON")
print("="*80)
print(f"\nTotal XLSX files: {len(xlsx_files)}")

# Group XLSX files by prefix (e.g., sdat2_data2020)
groups = {}
for f in xlsx_files:
    name = f['filename'].lower()
    # Extract prefix like "sdat2_data2020"
    match = re.match(r"^(sdat\d+_data\d+)_", name)
    if match:
        key = match.group(1)
    else:
        key = name.split('_')[0] if '_' in name else name.split('.')[0]
    groups.setdefault(key, []).append(f)

print(f"\nFound {len(groups)} groups\n")

for group_key in sorted(groups.keys()):
    group_files = groups[group_key]
    print("\n" + "="*80)
    print(f"Group: {group_key} ({len(group_files)} files)")
    
    # Get columns for each file
    schemas = {}
    for f in group_files:
        key = f['relative_path']
        schema = file_schemas.get(key, {})
        sheets = schema.get('sheets', [])
        if sheets:
            # Get columns from first sheet
            col_names = sheets[0].get('column_names', [])
            schemas[f['filename']] = col_names
    
    # Compare columns
    if len(group_files) > 1:
        base_file = group_files[0]['filename']
        base_cols = schemas.get(base_file, [])
        
        print(f"Baseline: {base_file}")
        print(f"  Columns: {len(base_cols)}")
        
        all_same = True
        for f in group_files[1:]:
            cols = schemas.get(f['filename'], [])
            if cols == base_cols:
                print(f"✅ {f['filename']}: Identical columns ({len(cols)})")
            else:
                all_same = False
                print(f"⚠️  {f['filename']}: Different columns ({len(cols)})")
                missing = [c for c in base_cols if c not in cols]
                extra = [c for c in cols if c not in base_cols]
                if missing:
                    print(f"     Missing: {missing}")
                if extra:
                    print(f"     Extra: {extra}")
        
        if all_same:
            print(f"\n✅ All {len(group_files)} files in this group have identical columns!")
    else:
        f = group_files[0]
        cols = schemas.get(f['filename'], [])
        print(f"{f['filename']}: {len(cols)} columns")

📊 XLSX FILE COLUMN COMPARISON

Total XLSX files: 2

Found 1 groups


Group: sdat2_data2020 (2 files)
Baseline: sdat2_data2020_delomr.xlsx
  Columns: 361
⚠️  sdat2_data2020_grunnkrets.xlsx: Different columns (361)
     Missing: ['Delområde']
     Extra: ['grunnkrets']
